In [25]:
from braindecode.preprocessing import (
    exponential_moving_standardize,
    preprocess,
    Preprocessor)
from numpy import multiply
from braindecode.datasets import MOABBDataset
from braindecode.preprocessing import create_windows_from_events
import torch
from braindecode.util import set_random_seeds

from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pandas as pd
from sklearn.metrics import confusion_matrix
from braindecode.visualization import plot_confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import sys
from mne.datasets import eegbci
import glob
from IPython.display import clear_output
import numpy as np
import torch
from torch import nn
import torch.optim as optim

from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from scipy import signal


import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math
import wandb
import os
import neptune.new.integrations.sklearn as npt_utils
import neptune.new as neptune

from sklearn.datasets import make_swiss_roll
from braindecode.models import ShallowFBCSPNet, Deep4Net ,EEGNetv4,HybridNet,EEGInceptionMI,EEGITNet,ATCNet
os.environ['http_proxy'] = "http://192.41.170.23:3128"
os.environ['https_proxy'] = "http://192.41.170.23:3128"
from moabb.datasets import BNCI2014001
from moabb.paradigms import MotorImagery


In [26]:
def create_dataloader(X, y, batch_size):
    X_tensor = torch.tensor(X).float()
    y_tensor = torch.tensor(y).long()
    dataset_tensor = TensorDataset(X_tensor, y_tensor)
    dl = torch.utils.data.DataLoader(dataset_tensor, batch_size=batch_size, shuffle=True)
    return dl

def early_stopping(train_loss, validation_loss, min_delta, tolerance):
    counter = 0
    if (validation_loss - train_loss) > min_delta:
        counter +=1
        if counter >= tolerance:
          return True

def extrack_dataset(dataset):
    for x, y, window_ind in dataset:
        x_shape = x.shape
        y_shape = len(dataset.get_metadata().target)
        break
    X = np.zeros((y_shape,x_shape[0],x_shape[1]))
    y_=[]
    i=0
    for x, y, window_ind in dataset:
        X[i]=x
        y_.append(y)
        i+=1
    #X2 = X[:, 7:8, :]
    #X3= X[:, 11:12, :]
    #(288, 22, 1125)
    #X = np.concatenate((X2,X3), axis=1)
    print(X.shape)

    #X = X.reshape(y_shape,X.shape[2],X.shape[1])
    return X,np.array(y_).T


def extrack_dataset_2class(dataset):
    for x, y, window_ind in dataset:
        x_shape = x.shape
        y_shape = len(dataset.get_metadata().target)
        break
    X = np.zeros((y_shape,x_shape[0],x_shape[1]))
    y_=[]
    i=0
    for x, y, window_ind in dataset:
        X[i]=x
        y_.append(y)
        i+=1
    y = np.array(y_).T
    X_ = np.zeros((int(y_shape/2) ,x_shape[0],x_shape[1])) #for 2 class
    y_for_2class =[]
    j=0
    for i in range(y.shape[0]):
        if y[i] == 0 or y[i] == 1:
            y_for_2class.append(y[i])
            X_[j]=X[i]
            j +=1
    y_for_2class = np.array(y_for_2class).T
    X_ = X_.reshape(y_for_2class.shape[0],X_.shape[2],X_.shape[1])
    return X_,y_for_2class

def extrack_dataset_2class_cut(dataset):
    for x, y, window_ind in dataset:
        x_shape = x.shape
        y_shape = len(dataset.get_metadata().target)
        break
    X = np.zeros((y_shape,x_shape[0],x_shape[1]))
    y_=[]
    i=0
    for x, y, window_ind in dataset:
        X[i]=x
        y_.append(y)
        i+=1
    y = np.array(y_).T
    X_ = np.zeros((int(y_shape/2) ,x_shape[0],x_shape[1])) #for 2 class
    y_for_2class =[]
    j=0
    for i in range(y.shape[0]):
        if y[i] == 0 or y[i] == 1:
            y_for_2class.append(y[i])
            X_[j]=X[i]
            j +=1
    y_for_2class = np.array(y_for_2class).T
    X_ = X_.reshape(y_for_2class.shape[0],X_.shape[2],X_.shape[1])
    #100, 1, 576, 22
    X_ = X_[:,64:320,:]
    print(X_.shape)
    return X_,y_for_2class


def train(model,gpu_num,train_loader,test_loader,
          weights_name=False,
          optimizer = None,
          criterion = None,
          num_epochs=500,
          vail_loader= None,
          save_weights = False,
          neptune = True,
          lr = None
         ):

    # Train the model


    train_loss = []
    valid_loss = [10,11]
    train_accuracy = []
    valid_accuracy = []

    old_loss = 100
    old_acc = 0
    valid_loss_vail = []


    for epoch in range(num_epochs):
        iter_loss = 0.0
        correct = 0
        iterations = 0

        model.train()

        for i, (items, classes) in enumerate(train_loader):
            items = Variable(items)
            classes = classes.type(torch.LongTensor)
            classes = Variable(classes)

            if cuda.is_available():
                items = items.cuda(gpu_num)
                classes = classes.cuda(gpu_num)

            optimizer.zero_grad()
            outputs = model(items)
            #avg_pedic = torch.mean(outputs, 1, True).float()
            #avg_pedic = avg_pedic.reshape(classes.shape[0])
            #print(avg_pedic.shape)
            #print(classes.shape)
            loss = criterion(outputs, classes)

            iter_loss += loss.item()
            loss.backward()
            optimizer.step()

            metrics = {"train/train_loss": loss}

            #print(loss)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == classes.data).sum()
            iterations += 1

        train_loss.append(iter_loss/iterations)


        train_accuracy.append(( correct.float() / len(train_loader.dataset)))
        train_metrics = {"train/train_loss": iter_loss/iterations,
                       "train/train_accuracy": (100 * correct.float() / len(train_loader.dataset))}



        loss = 0.0
        correct = 0
        iterations = 0
        model.eval()

        for i, (items, classes) in enumerate(test_loader):
            classes = classes.type(torch.LongTensor)
            items = Variable(items)
            classes = Variable(classes)

            if cuda.is_available():
                items = items.cuda(gpu_num)
                classes = classes.cuda(gpu_num)


            outputs = model(items)
            #avg_pedic = torch.mean(outputs, 1, True).float()
            #avg_pedic = avg_pedic.reshape(classes.shape[0])

            loss += criterion(outputs, classes).item()

            _, predicted = torch.max(outputs.data, 1)

            correct += (predicted == classes.data).sum()
            #print("correct : {}".format(classes.data))
            #print("predicted : {}".format(predicted))
            iterations += 1

        valid_loss.append(loss/iterations)
        correct_scalar = np.array([correct.clone().cpu()])[0]
        valid_accuracy.append(correct_scalar / len(test_loader.dataset) )

        test_metrics = {"Test/Test_loss": loss/iterations,
                       "Test/Test_accuracy": correct_scalar / len(test_loader.dataset) }
        if save_weights is True:
            if epoch+1 > 2 and valid_loss[-1] < old_loss and old_acc <= valid_accuracy[-1] :
                    newpath = r'./{}'.format(weights_name)
                    if not os.path.exists(newpath):
                        os.makedirs(newpath)
                    torch.save(model.state_dict(),'./{}/{:.4f}_{}_{:.4f}_{:.4f}'.format(weights_name,valid_loss[-1],weights_name,valid_loss[-1],valid_accuracy[-1]))
                    part_weights = './{}/{:.4f}_{}_{:.4f}_{:.4f}'.format(weights_name,valid_loss[-1],weights_name,valid_loss[-1],valid_accuracy[-1])
                    old_loss = valid_loss[-1]
                    old_acc = valid_accuracy[-1]

        print ('Epoch %d/%d, Tr Loss: %.4f, Tr Acc: %.4f, Val Loss: %.4f, Val Acc: %.4f , le : %f'
                       %(epoch+1, num_epochs, train_loss[-1], train_accuracy[-1], valid_loss[-1], valid_accuracy[-1],lr))
        if early_stopping(train_loss[-1], valid_loss[-1], min_delta=10, tolerance = 20):
            print("We are at epoch:", epoch+1)
            run[f"epoch/valid_accuracy"].append(valid_accuracy[-1])
            run[f"epoch/train_accuracy"].append(train_accuracy[-1])
            run[f"epoch/train_loss"].append(train_loss[-1])
            run[f"epoch/valid_loss"].append(valid_loss[-1])

            break
        if neptune is True:
            run[f"epoch/valid_accuracy"].append(valid_accuracy[-1])
            run[f"epoch/train_accuracy"].append(train_accuracy[-1])
            run[f"epoch/train_loss"].append(train_loss[-1])
            run[f"epoch/valid_loss"].append(valid_loss[-1])


    return train_loss,valid_loss,train_accuracy,valid_accuracy,part_weights
def eval(model,
         gpu_num,
          vail_loader= None,
         labels=None,
         ):
    model.eval()
    y_pred = []
    y_true = []
    correct=0
    for i, (items, classes) in enumerate(vail_loader):
        classes = classes.type(torch.LongTensor)
        items = Variable(items)
        classes = Variable(classes)
        if cuda.is_available():
            items = items.cuda(gpu_num)
            classes = classes.cuda(gpu_num)

        outputs = model(items)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.append(predicted.clone().cpu().numpy())
        y_true.append(classes.data.clone().cpu().numpy())
        correct += (predicted == classes.data).sum()
    correct_scalar = np.array([correct.clone().cpu()])[0]
    valid_accuracy.append(correct_scalar / len(test_loader.dataset) )

    confusion_mat = confusion_matrix(np.array(y_true).T,np.array(y_pred).T )
    run[f"epoch/eval_ACC"].append(valid_accuracy[-1])
    run["confusion matrices subject_id : {0}".format(subject_id)].upload(plot_confusion_matrix(confusion_mat, class_names=labels,rotate_row_labels=0,rotate_col_labels=90,with_f1_score=True))
    return y_pred,y_true,correct_scalar,valid_accuracy

In [30]:
n_steps = 1000
#types_list = ['left','right','foot','tongue',]
types_list = ['left','right','foot','tongue']

#subject_list = list(range(2,10))
subject_list = [1]
size_gen_list = [72]

#   288 , 22 , 1125
for size_gen in size_gen_list :
    print("size_gen : {}".format(size_gen))
    y = np.zeros((size_gen*(len(types_list)),))
    x_seq = np.zeros((size_gen*(len(types_list)),22,1125))
    for subject in subject_list:
        for types_num in range(len(types_list)):

            types = types_list[types_num]
            print("types  : {0} of {1}".format(types , subject))
            #print("types_num : {}".format(types_num))
            for i in range(22):
                for j in range(size_gen):
                    model = ConditionalModel(n_steps).cuda(0)
                    #print('diffusion/{0}_{1}_{2}'.format(subject,i,types))
                    model.load_state_dict(torch.load('diffusion_more/{0}_{1}_{2}'.format(subject,i,types)))
                    cur_x = torch.randn([1, 1125]).cuda(0)
                    cur_x = p_sample(model, cur_x,998)
                    #cur_x.shape
                    if types_num == 0 :
                        x_seq[j][i]= cur_x.cpu().detach().numpy()
                        if types == 'left':
                            y[j] = 0
                        if types == 'right':
                            y[j] = 1
                        if types == 'foot':
                            y[j] = 2
                        if types == 'tongue':
                            y[j] = 3



                        #print("types_num : {}".format(types_num))
                    if types_num == 1 or types_num == 2 or types_num == 3:
                        x_seq[(size_gen*types_num)+j][i]= cur_x.cpu().detach().numpy()
                        y[(size_gen*types_num)+j] = types_num
        np.save("Synthetic_data_more/X_subject_{0}_{1}_{2}.npy".format(subject,size_gen,types_list),x_seq)
        np.save("Synthetic_data_more/y_subject_{0}_{1}_{2}.npy".format(subject,size_gen,types_list),y)
        print("Done Synthetic data of subject {}".format(subject))


size_gen : 72
types  : left of 1
types  : right of 1
types  : foot of 1
types  : tongue of 1
Done Synthetic data of subject 1


##############################################

for size_gen in size_gen_list :
    print("size_gen : {}".format(size_gen))
    y = np.zeros((size_gen*4,))
    x_seq = np.zeros((size_gen*4,22,1125))
    for subject in subject_list:
        for types_num in range(1):
            types = types_list[types_num]
            print("types  : {0} of {1}".format(types , subject))
            #print("types_num : {}".format(types_num))
            for i in range(22):
                for j in range(size_gen):
                    model = ConditionalModel(n_steps).cuda(0)
                    print('diffusion/{0}_{1}_{2}'.format(subject,i,types))
                    model.load_state_dict(torch.load('diffusion/{0}_{1}_{2}'.format(subject,i,types)))

# only 1 CH


n_steps = 1000
#types_list = ['left','right','foot','tongue',]
types_list = ['foot']

subject_list = [3]#list(range(1,10))

size_gen_list = [72]#[18,36,54,72]

#   288 , 22 , 1125
for size_gen in size_gen_list :
    print("size_gen : {}".format(size_gen))
    y = np.zeros((size_gen*(len(types_list)),))
    x_seq = np.zeros((size_gen*(len(types_list)),1,1125))
    for subject in subject_list:
        for types_num in range(len(types_list)):

            types = types_list[types_num]
            print("types  : {0} of {1}".format(types , subject))
            #print("types_num : {}".format(types_num))
            for i in [0]:#range(22):
                for j in range(size_gen):
                    model = ConditionalModel(n_steps).cuda(0)
                    #print('diffusion/{0}_{1}_{2}'.format(subject,i,types))
                    model.load_state_dict(torch.load('diffusion/{0}_{1}_{2}'.format(subject,i,types)))
                    cur_x = torch.randn([1, 1125]).cuda(0)
                    cur_x = p_sample(model, cur_x,998)
                    #cur_x.shape
                    if types_num == 0 :
                        x_seq[j][i]= cur_x.cpu().detach().numpy()
                        y[j] = types_num

                        #print("types_num : {}".format(types_num))
                    if types_num == 1 or types_num == 2 or types_num == 3:
                        x_seq[(size_gen*types_num)+j][i]= cur_x.cpu().detach().numpy()
                        y[(size_gen*types_num)+j] = types_num
        np.save("Synthetic_data_all/X_subject_{0}_{1}_{2}_1ch.npy".format(subject,size_gen,types_list),x_seq)
        #np.save("Synthetic_data_all/y_subject_{0}_{1}_{2}.npy".format(subject,size_gen,types_list),y)
        print("Done Synthetic data of subject {}".format(subject))


In [31]:
from ignite.metrics import FID, InceptionScore
import os
os.environ['http_proxy'] = "http://192.41.170.23:3128"
os.environ['https_proxy'] = "http://192.41.170.23:3128"

In [32]:
from collections import OrderedDict

import torch
from torch import nn, optim

from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.utils import *
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *

# create default evaluator for doctests

def eval_step(engine, batch):
    return batch

default_evaluator = Engine(eval_step)

# create default optimizer for doctests

param_tensor = torch.zeros([1], requires_grad=True)
default_optimizer = torch.optim.SGD([param_tensor], lr=0.1)

# create default trainer for doctests
# as handlers could be attached to the trainer,
# each test must define his own trainer using `.. testsetup:`

def get_default_trainer():

    def train_step(engine, batch):
        return batch

    return Engine(train_step)

# create default model for doctests

default_model = nn.Sequential(OrderedDict([
    ('base', nn.Linear(4, 2)),
    ('fc', nn.Linear(2, 1))
]))

manual_seed(666)

In [33]:
import numpy as np

In [34]:
X=np.load("Synthetic_data_all/X_subject_1_72_['foot']_1ch.npy")
t = torch.from_numpy(X)
y = torch.rand(10, 3, 299, 299)

In [35]:
t.shape

torch.Size([72, 1, 1125])

In [36]:
metric = InceptionScore()
metric.attach(default_evaluator, "is")

state = default_evaluator.run([t])
print(state.metrics["is"])

Current run is terminating due to exception: Inputs should be a tensor of dim 4, got 3
Engine run is terminating due to exception: Inputs should be a tensor of dim 4, got 3


ValueError: Inputs should be a tensor of dim 4, got 3

In [ ]:
metric = InceptionScore(num_features=1, feature_extractor=default_model)
metric.attach(default_evaluator, "is")
y = torch.zeros(10, 4)
state = default_evaluator.run([y])
print(state.metrics["is"])